# 预处理文本数据

In [1]:
from pathlib import Path
import pandas as pd
import re
import sqlite3
# import openpyxl


## 配置路径

In [2]:
# 配置数据集基本路径
folderpath_original_data_root = Path.cwd().parent.parent.parent.joinpath('EntelechyData/original')
Path.mkdir(folderpath_original_data_root, exist_ok=True)

folderpath_pretreat_data_root = Path.cwd().parent.parent.parent.joinpath('EntelechyData/pretreatment')
Path.mkdir(folderpath_pretreat_data_root, exist_ok=True)

# 配置数据集路径
folderpath_pretreat_现代汉语词典 = Path(folderpath_pretreat_data_root, f'现代汉语词典')
folderpath_pretreat_新华字典 = Path(folderpath_pretreat_data_root, f'新华字典')
folderpath_pretreat_现代汉语常用字符集 = Path(folderpath_pretreat_data_root, f'现代汉语常用字符集')


## 导入预处理的词库数据

### 导入 sqlite 数据库

In [13]:
# 导入 sqlite 数据库
conn_现代汉语词典 = sqlite3.connect(Path(folderpath_pretreat_现代汉语词典, '现代汉语词典.db'))
conn_新华字典 = sqlite3.connect(Path(folderpath_pretreat_新华字典, '新华字典.db'))


### 导入 pandas 数据

In [3]:
# 读取新华字典 pandas 数据库内容
df_新华字典 = pd.read_pickle(Path(folderpath_pretreat_新华字典, '新华字典.pkl'))
# 读取现代汉语词典 pandas 数据库内容
df_现代汉语词典 = pd.read_pickle(Path(folderpath_pretreat_现代汉语词典, '现代汉语词典.pkl'))
df_现代汉语常用字符集 = pd.read_pickle(Path(folderpath_pretreat_现代汉语常用字符集, '现代汉语常用字符集.pkl'))

## 导入自定义的初始的基础概念库 

### 初始化自定义的初始的基础概念库

In [16]:
# 初始化自定义的初始的基础概念库
df_基础概念_现代汉语字符库 = pd.DataFrame(
    columns=['uid', 'id_概念', 'name_概念', '内容_010', '基础类别', '是否常用字', '备注'])
df_基础概念_现代汉语词库 = pd.DataFrame(columns=['uid', 'id_概念', 'name_概念', '内容_010', '是否常用词', '备注'])

df_基础概念_现代汉语字符库['内容_010'] = df_新华字典['word']  # 导入新华字典 pandas 数据库内容到自定义的初始的基础概念库
df_基础概念_现代汉语字符库['基础类别'] = '汉字集'
df_基础概念_现代汉语字符库['是否常用字'] = '否'


### 【df_基础概念_现代汉语字符库】 标记常用字

In [17]:
# 标记常用字
for index, row in df_现代汉语常用字符集.iterrows():
    if row['类别'] == '汉字集':
        if row['字符'] in df_基础概念_现代汉语字符库['内容_010'].values:
            index_02 = df_基础概念_现代汉语字符库[df_基础概念_现代汉语字符库['内容_010'] == row['字符']].index[0]
            df_基础概念_现代汉语字符库.loc[index_02, '是否常用字'] = '是'
            df_基础概念_现代汉语字符库.loc[index_02, '备注'] = f"现代汉语常用字【{row['字符']}】"


In [20]:
df_基础概念_现代汉语字符库 = df_基础概念_现代汉语字符库.copy()
# 插入非汉字集的字符
for index, row in df_现代汉语常用字符集.iterrows():
    if row['类别'] != '汉字集':
        df_基础概念_现代汉语字符库 = pd.concat(
            [
                df_基础概念_现代汉语字符库,
                pd.DataFrame({
                    '内容_010': [row['字符']],
                    '基础类别': [row['类别']],
                    '是否常用字': ['是'],
                    '备注': row['备注'],
                })
            ]
        )


In [ ]:
# 添加备注


In [21]:
df_基础概念_现代汉语字符库['基础类别'].unique()

array(['汉字集', '中文字符集', '英文标点符号和字符集', '英文特殊符号集', '英文字符集', '数字集'],
      dtype=object)

In [22]:
# 自定义基础类别的排序顺序

category_order = ['数字集', '英文字符集', '英文特殊符号集', '英文标点符号和字符集', '中文字符集', '汉字集']

# 按照是否常用字排序。优先按照基础类别排序，再按照是否常用字排序。其中基础类别按照自定义顺序排序
df_基础概念_现代汉语字符库['基础类别'] = pd.Categorical(df_基础概念_现代汉语字符库['基础类别'],
                                                        categories=category_order, ordered=True)
df_基础概念_现代汉语字符库 = df_基础概念_现代汉语字符库.sort_values(by=['基础类别', '是否常用字'],
                                                                    ascending=[True, False])

In [23]:
df_基础概念_现代汉语字符库['id_概念'] = df_基础概念_现代汉语字符库.index
df_基础概念_现代汉语字符库['uid'] = df_基础概念_现代汉语字符库.index

df_基础概念_现代汉语字符库.set_index('uid', inplace=True)  # 设置 index


### 导出现代汉语字符库原始概念库

In [24]:
folderpath_original_conceptions = folderpath_pretreat_data_root = Path.cwd().parent.parent.parent.joinpath(
    'EntelechyData/conceptions')
Path.mkdir(folderpath_original_conceptions, exist_ok=True)
# 导出为 pickle 文件
df_基础概念_现代汉语字符库.to_pickle(Path(folderpath_original_conceptions, '基础概念_现代汉语字符库.pkl'))
# 导出为 csv 文件
df_基础概念_现代汉语字符库.to_csv(Path(folderpath_original_conceptions, '基础概念_现代汉语字符库.csv'))
# # 导出为 sqlite 数据库
# df_基础概念_现代汉语字符库.to_sql('基础概念_现代汉语字符库', conn_现代汉语词典, if_exists='replace', index=True)
# # 导出为 xlsx 文件
# df_基础概念_现代汉语字符库.to_excel(Path(folderpath_original_conceptions, '基础概念_现代汉语字符库.xlsx'), index=True)

### 初始化基础概念库的现代汉语词库（后续在CIS处理）

In [22]:
# # 初始化自定义的初始的基础概念库
# df_基础概念_现代汉语字符库 = pd.DataFrame(
#     columns=['uid', 'id_概念', 'name_概念', '内容_010', '基础类别', '是否常用字', '备注'])
# df_基础概念_现代汉语词库 = pd.DataFrame(columns=['uid', 'id_概念', 'name_概念', '内容_010', '是否常用词', '备注'])

# df_基础概念_现代汉语字符库['内容_010'] = df_新华字典['word']  # 导入新华字典 pandas 数据库内容到自定义的初始的基础概念库
# df_基础概念_现代汉语字符库['基础类别'] = '汉字集'
# df_基础概念_现代汉语字符库['是否常用字'] = '否'
